In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Masked VAE

In [2]:
import pandas as pd,numpy as np

In [3]:
df=pd.read_csv("/content/drive/MyDrive/Train/data/NewFullInsuranceFraudData.csv",low_memory=False)
# df=df.iloc[:,1:]

In [4]:
"IsFraud" in df.columns.tolist()

True

In [5]:
"TransactionID" in df.columns.tolist()

True

In [6]:
print(df.columns.tolist())

['AgeOfVehicle_7 years', 'dataset_name', 'NoOfMonths_PartACov_11', 'NumberOfCars_3 to 4', 'RenalDiseaseIndicator', 'Days:Policy-Accident_1 to 7', 'PolicyType_Sedan - Collision', 'umbrella_limit_0', 'SOCIAL_CLASS_MI', 'MaritalStatus_Single', 'PastNumberOfClaims_more than 4', 'NoOfMonths_PartACov_8', 'AgeOfVehicle_new', 'ChronicCond_Osteoporasis_2', 'witnesses_1', 'DriverRating_1', 'CITY_Nashville', 'number_of_vehicles_involved_1', 'incident_state_SC', 'INCIDENT_CITY_Manchester', 'CUSTOMER_EDUCATION_LEVEL_High School', 'VehiclePrice_20,000 to 29,000', 'incident_city_Arlington', 'STATE_AR', 'ChronicCond_ObstrPulmonary_2', 'insured_hobbies_bungie-jumping', 'AUTHORITY_CONTACTED_Police', 'auto_model_MDX', 'authorities_contacted_Police', 'DayOfWeekClaimed_Monday', 'WeekOfMonth_5', 'INCIDENT_CITY_Glendale', 'insured_education_level_JD', 'insured_relationship_unmarried', 'Police_Report', 'CUSTOMER_EDUCATION_LEVEL_Bachelor', 'MonthClaimed_Mar', 'insured_education_level_Associate', 'IPAnnualReimb

In [7]:
def display(matching_rows):
    print("counting list of Uniquic all featurs")
    li=[]
    for i in matching_rows.columns:
        #print(str(i),' :',len(matching_rows[i].unique()),matching_rows[i].unique())
        print(str(i), ' :', matching_rows[i].nunique(),',',list(matching_rows[i].unique())[:2])
        li.append(matching_rows[i].nunique())
    #return li
        # if(matching_rows[i].nunique()<50):
        #     print(str(i),' :',matching_rows[i].unique())

In [8]:
display(df)

counting list of Uniquic all featurs
AgeOfVehicle_7 years  : 2 , [np.float64(nan), np.float64(0.0)]
dataset_name  : 4 , ['Insurance Fraud Detection', 'Insurance Claims Fraud Data']
NoOfMonths_PartACov_11  : 2 , [np.float64(nan), np.float64(0.0)]
NumberOfCars_3 to 4  : 2 , [np.float64(nan), np.float64(1.0)]
RenalDiseaseIndicator  : 2 , [np.float64(nan), np.float64(0.0)]
Days:Policy-Accident_1 to 7  : 2 , [np.float64(nan), np.float64(0.0)]
PolicyType_Sedan - Collision  : 2 , [np.float64(nan), np.float64(0.0)]
umbrella_limit_0  : 2 , [np.float64(1.0), np.float64(0.0)]
SOCIAL_CLASS_MI  : 2 , [np.float64(nan), np.float64(0.0)]
MaritalStatus_Single  : 2 , [np.float64(nan), np.float64(1.0)]
PastNumberOfClaims_more than 4  : 2 , [np.float64(nan), np.float64(0.0)]
NoOfMonths_PartACov_8  : 2 , [np.float64(nan), np.float64(0.0)]
AgeOfVehicle_new  : 2 , [np.float64(nan), np.float64(0.0)]
ChronicCond_Osteoporasis_2  : 2 , [np.float64(nan), np.float64(1.0)]
witnesses_1  : 2 , [np.float64(0.0), np.fl

In [9]:
df.shape

(584631, 281)

In [10]:
df.value_counts("IsFraud")

,count
IsFraud,
0.0,370162
1.0,214469


In [11]:
df['IsFraud'].value_counts()/len(df)

,count
IsFraud,
0.0,0.633155
1.0,0.366845


In [12]:
# a =df.copy()

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 584631 entries, 0 to 584630
Columns: 281 entries, AgeOfVehicle_7 years to Days:Policy-Claim_none
dtypes: float64(278), int64(1), object(2)
memory usage: 1.2+ GB


In [44]:
# import pandas as pd

# # Set option to show all columns when printing
# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)      # If needed for rows
# pd.set_option('display.max_colwidth', None)  # Show full content in cells

# # Now print all dtypes
# print(df.dtypes)


In [14]:
# Show only columns with non-numeric data types
non_numeric_cols = df.select_dtypes(exclude=['int64', 'float64']).dtypes

print(non_numeric_cols)


dataset_name     object
TransactionID    object
dtype: object


In [15]:
"object" in df.dtypes.tolist()

True

In [16]:
df.drop(columns =["TransactionID","dataset_name"],inplace=True)

In [17]:
# threshold = 0.98
# cols_to_drop = [col for col in df.columns if df[col].isna().mean() > threshold]
# df = df.drop(columns=cols_to_drop)


In [18]:
df.shape#131 features are their

(584631, 279)

In [19]:
df.shape

(584631, 279)

In [20]:
def print_null_counts(df):
    null_counts = df.isnull().sum()
    total_rows = len(df)

    print(f"Total rows: {total_rows}\n")
    print(f"{'Column Name':35} | {'Null Count':10} | {'Null Percentage':15}")
    print("-" * 65)

    for col, null_count in null_counts.items():
        null_pct = (null_count / total_rows) * 100
        print(f"{col:35} | {null_count:<10} | {null_pct:>14.2f}%")


In [16]:
print_null_counts(df)

Total rows: 584631

Column Name                         | Null Count | Null Percentage
-----------------------------------------------------------------
AgeOfVehicle_7 years                | 569211     |          97.36%
NoOfMonths_PartACov_11              | 26420      |           4.52%
NumberOfCars_3 to 4                 | 569211     |          97.36%
RenalDiseaseIndicator               | 26420      |           4.52%
Days:Policy-Accident_1 to 7         | 569211     |          97.36%
PolicyType_Sedan - Collision        | 569211     |          97.36%
umbrella_limit_0                    | 583631     |          99.83%
SOCIAL_CLASS_MI                     | 574631     |          98.29%
MaritalStatus_Single                | 569211     |          97.36%
PastNumberOfClaims_more than 4      | 569211     |          97.36%
NoOfMonths_PartACov_8               | 26420      |           4.52%
AgeOfVehicle_new                    | 569211     |          97.36%
ChronicCond_Osteoporasis_2          | 26420

In [22]:
# for i in df.columns:
#     print(i)
df_new=df

In [21]:
df.shape

(584631, 279)

In [23]:
# 1. Mask: 1 for observed, 0 for missing
mask = (~df.isna()).astype('float32')

# 2. Compute observed mean & std (ignore missing)
df_mean = df.sum() / mask.sum()
df_var = ((df - df_mean) ** 2 * mask).sum() / mask.sum()
df_std = np.sqrt(df_var + 1e-6)

# 3. Normalize (skip missing entries)
df_normalized = (df - df_mean) / df_std

# 4. Fill NaNs with 0 for model input
df_filled = df_normalized.fillna(0.0)

# 5. Convert to NumPy
data_np = df_filled.to_numpy().astype('float32')
mask_np = mask.to_numpy().astype('float32')


In [24]:
import numpy as np
import pandas as pd
from tensorflow.keras.layers import Input, Dense, Lambda, Layer
from tensorflow.keras.models import Model
from tensorflow.keras.losses import binary_crossentropy
from tensorflow.keras import backend as K

# Custom layer for VAE loss
class VAELossLayer(Layer):
    def __init__(self, **kwargs):
        super(VAELossLayer, self).__init__(**kwargs)

    def vae_loss(self, x, x_decoded_mean, z_log_var, z_mean):
        reconstruction_loss = binary_crossentropy(x, x_decoded_mean)
        reconstruction_loss *= original_dim
        kl_loss = 1 + z_log_var - K.square(z_mean) - K.exp(z_log_var)
        kl_loss = K.sum(kl_loss, axis=-1)
        kl_loss *= -0.5
        return K.mean(reconstruction_loss + kl_loss)

    def call(self, inputs):
        x = inputs[0]
        x_decoded_mean = inputs[1]
        z_log_var = inputs[2]
        z_mean = inputs[3]

        loss = self.vae_loss(x, x_decoded_mean, z_log_var, z_mean)
        self.add_loss(loss)

        return x

class MaskedVAELoss(Layer):
    def call(self, inputs):
        x, x_decoded, z_mean, z_log_var, mask = inputs

        # Clip z_log_var to avoid exploding exp()
        z_log_var = K.clip(z_log_var, -10.0, 10.0)

        # Apply per-sample masking to reconstruction loss
        squared_diff = K.square(x - x_decoded)
        masked_squared_diff = mask * squared_diff

        recon_loss = K.sum(masked_squared_diff, axis=-1)
        recon_loss /= (K.sum(mask, axis=-1) + K.epsilon())  # avoid div by zero

        # KL divergence
        kl = -0.5 * K.sum(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)

        total_loss = K.mean(recon_loss + kl)
        self.add_loss(total_loss)

        return x  # return input, we only care about loss


In [25]:
from tensorflow.keras.layers import Input, Dense, Lambda
from tensorflow.keras.models import Model
original_dim = data_np.shape[1]
intermediate_dim = 64
latent_dim = 32

# Inputs
x_input = Input(shape=(original_dim,), name='input_data')
mask_input = Input(shape=(original_dim,), name='input_mask')  # <- New

# Encoder
h = Dense(intermediate_dim, activation='relu')(x_input)
z_mean = Dense(latent_dim)(h)
z_log_var = Dense(latent_dim)(h)

def sampling(args):
    mean, log_var = args
    epsilon = K.random_normal(shape=K.shape(mean))
    return mean + K.exp(0.5 * log_var) * epsilon

z = Lambda(sampling)([z_mean, z_log_var])

# Decoder
decoder_h = Dense(intermediate_dim, activation='relu')
decoder_out = Dense(original_dim, activation='linear')
h_dec = decoder_h(z)
x_decoded = decoder_out(h_dec)

# Masked loss layer
loss_layer = MaskedVAELoss()([x_input, x_decoded, z_mean, z_log_var, mask_input])

# Define model with 2 inputs
vae = Model(inputs=[x_input, mask_input], outputs=loss_layer)
from tensorflow.keras.optimizers import Adam
vae.compile(optimizer=Adam(learning_rate=1e-4))  # Slower but stable

In [26]:
import numpy as np

print("NaN in data:", np.isnan(data_np).sum())
print("Inf in data:", np.isinf(data_np).sum())

print("NaN in mask:", np.isnan(mask_np).sum())
print("Inf in mask:", np.isinf(mask_np).sum())


NaN in data: 0
Inf in data: 0
NaN in mask: 0
Inf in mask: 0


In [27]:
from tensorflow.keras.callbacks import TerminateOnNaN


vae.fit([data_np, mask_np], epochs=3, batch_size=32,callbacks=[TerminateOnNaN()],validation_split=0.1)


Epoch 1/3
16443/16443 ━━━━━━━━━━━━━━━━━━━━ 57s 3ms/step - loss: 2.5112 - val_loss: 2.1613
Epoch 2/3
16443/16443 ━━━━━━━━━━━━━━━━━━━━ 55s 3ms/step - loss: 0.9992 - val_loss: 2.1130
Epoch 3/3
16443/16443 ━━━━━━━━━━━━━━━━━━━━ 56s 3ms/step - loss: 0.9880 - val_loss: 2.0932


In [28]:
# Step 1: Select the first 300 rows
df_subset = df_new.iloc[:300].copy()

# Step 2: Create missing value mask
missing_mask = df_subset.isnull()

# Step 3: Fill missing values with 0
data_filled = df_subset.fillna(0)

# Step 4: Convert to numpy arrays
data_filled_np = data_filled.to_numpy()
mask_np = (~missing_mask).astype(int).to_numpy()  # 1 if not missing, 0 if missing

# Step 5: Predict missing values using VAE
predicted_data = vae.predict([data_filled_np, mask_np])

# Step 6: Replace only missing values with predicted ones
imputed_array = np.where(missing_mask.to_numpy(), predicted_data, data_filled_np)

# Step 7: Denormalize imputed data
imputed_array = imputed_array * df_new.std().to_numpy() + df_new.mean().to_numpy()



10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step


In [38]:
def print_null_counts(df):
    null_counts = df.isnull().sum()
    total_rows = len(df)

    print(f"Total rows: {total_rows}\n")
    print(f"{'Column Name':35} | {'Null Count':10} | {'Null Percentage':15}")
    print("-" * 65)

    for col, null_count in null_counts.items():
        null_pct = (null_count / total_rows) * 100
        print(f"{col:35} | {null_count:<10} | {null_pct:>14.2f}%")
def display(matching_rows):
    print("counting list of Uniquic all featurs")
    li=[]
    for i in matching_rows.columns:
        #print(str(i),' :',len(matching_rows[i].unique()),matching_rows[i].unique())
        print(str(i), ' :', matching_rows[i].nunique(),',',list(matching_rows[i].unique())[:2])
        li.append(matching_rows[i].nunique())
    #return li
        # if(matching_rows[i].nunique()<50):
        #     print(str(i),' :',matching_rows[i].unique())
def print_categorical_value_counts(df, max_unique_values=10):
    """
    Prints structured value counts for each categorical feature in the DataFrame.
    Detects both object-type columns and numeric columns with low unique values.

    Parameters:
    - df: pandas DataFrame
    - max_unique_values: maximum number of unique values to consider a numeric column categorical

    Returns:
    - None (output is printed)
    """
    import pandas as pd

    # Step 1: Identify object or category dtype columns
    cat_cols = list(df.select_dtypes(include=['object', 'category']).columns)

    # Step 2: Add numeric columns with few unique values (likely categorical)
    for col in df.select_dtypes(include=['int64', 'int32', 'float64']).columns:
        num_unique = df[col].nunique(dropna=False)  # Count unique values including NaNs
        if num_unique <= max_unique_values:
            cat_cols.append(col)

    # Step 3: Deduplicate list of categorical-like columns
    cat_cols = list(set(cat_cols))

    # Step 4: Print structured value counts for each column
    for col in cat_cols:
        print(f"Feature: '{col}'")
        print(f" - Data Type      : {df[col].dtype}")
        print(f" - Non-Null Count : {df[col].notna().sum()}")
        print(f" - Null Count     : {df[col].isna().sum()}")
        print(f" - Unique Values  : {df[col].nunique(dropna=False)}")
        print("\nValue Counts:")
        print(df[col].value_counts(dropna=False))
        print("----------")

    return None


In [ ]:
# # Impute missing values
# missing_mask = df_new.isnull()
# data_filled = df_new.fillna(0)
# # Convert to numpy array for prediction
# data_filled_np = data_filled.to_numpy()

# predicted_data = vae.predict([data_filled_np, mask_np]) # Pass both data and mask

# # Replace missing values in original data with imputed values
# imputed_data = df_new.where(~missing_mask, predicted_data)

# # Denormalize to original scale
# imputed_data = imputed_data * df_new.std() + df_new.mean()

In [37]:
# display(imputed_data)

In [ ]:
# print_null_counts(imputed_data)

In [ ]:
# print_categorical_value_counts(imputed_data)

In [31]:
# print_null_counts(imputed_data)

In [ ]:
imputed_df_subset = pd.DataFrame(imputed_array, columns=df_new.columns, index=df_new.index[:300])


In [40]:
print_null_counts(df_new.iloc[:300])

Total rows: 300

Column Name                         | Null Count | Null Percentage
-----------------------------------------------------------------
AgeOfVehicle_7 years                | 300        |         100.00%
NoOfMonths_PartACov_11              | 300        |         100.00%
NumberOfCars_3 to 4                 | 300        |         100.00%
RenalDiseaseIndicator               | 300        |         100.00%
Days:Policy-Accident_1 to 7         | 300        |         100.00%
PolicyType_Sedan - Collision        | 300        |         100.00%
umbrella_limit_0                    | 0          |           0.00%
SOCIAL_CLASS_MI                     | 300        |         100.00%
MaritalStatus_Single                | 300        |         100.00%
PastNumberOfClaims_more than 4      | 300        |         100.00%
NoOfMonths_PartACov_8               | 300        |         100.00%
AgeOfVehicle_new                    | 300        |         100.00%
ChronicCond_Osteoporasis_2          | 300     

In [42]:
print_null_counts(imputed_df_subset)

Total rows: 300

Column Name                         | Null Count | Null Percentage
-----------------------------------------------------------------
AgeOfVehicle_7 years                | 0          |           0.00%
NoOfMonths_PartACov_11              | 0          |           0.00%
NumberOfCars_3 to 4                 | 0          |           0.00%
RenalDiseaseIndicator               | 0          |           0.00%
Days:Policy-Accident_1 to 7         | 0          |           0.00%
PolicyType_Sedan - Collision        | 0          |           0.00%
umbrella_limit_0                    | 0          |           0.00%
SOCIAL_CLASS_MI                     | 0          |           0.00%
MaritalStatus_Single                | 0          |           0.00%
PastNumberOfClaims_more than 4      | 0          |           0.00%
NoOfMonths_PartACov_8               | 0          |           0.00%
AgeOfVehicle_new                    | 0          |           0.00%
ChronicCond_Osteoporasis_2          | 0       

In [43]:
print_categorical_value_counts(imputed_df_subset)

Feature: 'Fault_Third Party'
 - Data Type      : float64
 - Non-Null Count : 300
 - Null Count     : 0
 - Unique Values  : 1

Value Counts:
Fault_Third Party
0.271725    300
Name: count, dtype: int64
----------
Feature: 'VehicleCategory_Sport'
 - Data Type      : float64
 - Non-Null Count : 300
 - Null Count     : 0
 - Unique Values  : 1

Value Counts:
VehicleCategory_Sport
0.347471    300
Name: count, dtype: int64
----------
Feature: 'HOUSE_TYPE_Mortgage'
 - Data Type      : float64
 - Non-Null Count : 300
 - Null Count     : 0
 - Unique Values  : 1

Value Counts:
HOUSE_TYPE_Mortgage
0.3652    300
Name: count, dtype: int64
----------
Feature: 'BasePolicy_Collision'
 - Data Type      : float64
 - Non-Null Count : 300
 - Null Count     : 0
 - Unique Values  : 1

Value Counts:
BasePolicy_Collision
0.386641    300
Name: count, dtype: int64
----------
Feature: 'BasePolicy_All Perils'
 - Data Type      : float64
 - Non-Null Count : 300
 - Null Count     : 0
 - Unique Values  : 1

Value Coun

In [ ]:
imputed_data.to_csv("/content/drive/MyDrive/Train/data/New_imputed_data.csv")

In [45]:
# # Optional: Save as a single HDF5 file
# vae.save("/content/drive/MyDrive/Train/data/my_model.h5")  # Creates file 'my_model.h5'
vae.save("/content/drive/MyDrive/Train/data/masked_vae1.keras")  # Recommended format


## Loading Model

In [ ]:
import pandas as pd
import numpy as np

df=pd.read_csv("/content/drive/MyDrive/Train/data/dataset_part2.csv",low_memory=False)


In [ ]:
a = df.copy()

In [ ]:
df.drop(columns =["TransactionID","dataset_name"],inplace=True)

In [ ]:
print(df.columns.tolist())

['AgeOfVehicle_7 years', 'NoOfMonths_PartACov_11', 'NumberOfCars_3 to 4', 'RenalDiseaseIndicator', 'Days:Policy-Accident_1 to 7', 'PolicyType_Sedan - Collision', 'umbrella_limit_0', 'SOCIAL_CLASS_MI', 'MaritalStatus_Single', 'PastNumberOfClaims_more than 4', 'NoOfMonths_PartACov_8', 'AgeOfVehicle_new', 'ChronicCond_Osteoporasis_2', 'witnesses_1', 'DriverRating_1', 'CITY_Nashville', 'number_of_vehicles_involved_1', 'incident_state_SC', 'INCIDENT_CITY_Manchester', 'CUSTOMER_EDUCATION_LEVEL_High School', 'VehiclePrice_20,000 to 29,000', 'incident_city_Arlington', 'STATE_AR', 'ChronicCond_ObstrPulmonary_2', 'insured_hobbies_bungie-jumping', 'AUTHORITY_CONTACTED_Police', 'auto_model_MDX', 'authorities_contacted_Police', 'DayOfWeekClaimed_Monday', 'WeekOfMonth_5', 'INCIDENT_CITY_Glendale', 'insured_education_level_JD', 'insured_relationship_unmarried', 'Police_Report', 'CUSTOMER_EDUCATION_LEVEL_Bachelor', 'MonthClaimed_Mar', 'insured_education_level_Associate', 'IPAnnualReimbursementAmt', 'C

In [ ]:
# for i in df.columns:
#     print(i)
df_new=df

In [ ]:
data_normalized = (df_new - df_new.mean()) / df_new.std()
# data_normalized


In [ ]:
# ----------------------------------------
# 2. Load your data (df_new and a)
# ----------------------------------------
# Make sure to replace this with your actual data loading logic

# Normalize data
data_normalized = (df_new - df_new.mean()) / df_new.std()
original_dim = data_normalized.shape[1]

In [ ]:
data_normalized_np = data_normalized.to_numpy()


In [ ]:
from tensorflow.keras.saving import register_keras_serializable
import numpy as np
from tensorflow.keras.layers import Input, Dense, Lambda, Layer
from tensorflow.keras.models import Model
from tensorflow.keras.losses import binary_crossentropy
from tensorflow.keras import backend as K
from tensorflow.keras.models import load_model

# Register again here too (for loading)
@register_keras_serializable()
def sampling(args):
    z_mean, z_log_var = args
    batch = K.shape(z_mean)[0]
    dim = K.int_shape(z_mean)[1]
    epsilon = K.random_normal(shape=(batch, dim))
    return z_mean + K.exp(0.5 * z_log_var) * epsilon

# Redefine VAELossLayer
class VAELossLayer(Layer):
    def __init__(self, **kwargs):
        super(VAELossLayer, self).__init__(**kwargs)

    def vae_loss(self, x, x_decoded_mean, z_log_var, z_mean):
        reconstruction_loss = binary_crossentropy(x, x_decoded_mean)
        reconstruction_loss *= original_dim
        kl_loss = 1 + z_log_var - K.square(z_mean) - K.exp(z_log_var)
        kl_loss = K.sum(kl_loss, axis=-1)
        kl_loss *= -0.5
        return K.mean(reconstruction_loss + kl_loss)

    def call(self, inputs):
        x = inputs[0]
        x_decoded_mean = inputs[1]
        z_log_var = inputs[2]
        z_mean = inputs[3]

        loss = self.vae_loss(x, x_decoded_mean, z_log_var, z_mean)
        self.add_loss(loss)
        return x

# Load the model with custom objects
vae = load_model(
    "/content/drive/MyDrive/Train/data/my_model.keras",
    custom_objects={
        "VAELossLayer": VAELossLayer,
        "sampling": sampling
    }
)


In [ ]:
# ----------------------------------------
# 4. Predict missing values
# ----------------------------------------
# Create a mask of missing values
missing_mask = data_normalized.isnull()

# Fill missing values with zeros for input
data_filled = data_normalized.fillna(0)

# Convert to numpy
data_filled_np = data_filled.to_numpy()

# Use the VAE to predict
predicted_data = vae.predict(data_filled_np)

# Replace missing values in original data
imputed_data = data_normalized.where(~missing_mask, predicted_data)

# Denormalize to original scale
imputed_data = imputed_data * df_new.std() + df_new.mean()




9135/9135 ━━━━━━━━━━━━━━━━━━━━ 13s 1ms/step


In [ ]:
# Round for cleaner output
imputed_data = imputed_data.round(3)


In [32]:
def display(matching_rows):
    print("counting list of Uniquic all featurs")
    li=[]
    for i in matching_rows.columns:
        #print(str(i),' :',len(matching_rows[i].unique()),matching_rows[i].unique())
        print(str(i), ' :', matching_rows[i].nunique(),',',list(matching_rows[i].unique())[:2])
        li.append(matching_rows[i].nunique())
    #return li
        # if(matching_rows[i].nunique()<50):
        #     print(str(i),' :',matching_rows[i].unique())

In [ ]:
display(imputed_data)

counting list of Uniquic all featurs
AgeOfVehicle_7 years  : 3 , [np.float64(0.377), np.float64(0.0)]
NoOfMonths_PartACov_11  : 2 , [np.float64(0.0), np.float64(1.0)]
NumberOfCars_3 to 4  : 3 , [np.float64(0.024), np.float64(1.0)]
RenalDiseaseIndicator  : 3 , [np.float64(0.0), np.float64(1.0)]
Days:Policy-Accident_1 to 7  : 3 , [np.float64(0.001), np.float64(0.0)]
PolicyType_Sedan - Collision  : 3 , [np.float64(0.362), np.float64(0.0)]
umbrella_limit_0  : 0 , [np.float64(nan)]
SOCIAL_CLASS_MI  : 0 , [np.float64(nan)]
MaritalStatus_Single  : 3 , [np.float64(0.304), np.float64(1.0)]
PastNumberOfClaims_more than 4  : 3 , [np.float64(0.13), np.float64(0.0)]
NoOfMonths_PartACov_8  : 2 , [np.float64(0.0), np.float64(1.0)]
AgeOfVehicle_new  : 3 , [np.float64(0.024), np.float64(0.0)]
ChronicCond_Osteoporasis_2  : 3 , [np.float64(1.0), np.float64(0.0)]
witnesses_1  : 0 , [np.float64(nan)]
DriverRating_1  : 3 , [np.float64(0.256), np.float64(1.0)]
CITY_Nashville  : 0 , [np.float64(nan)]
number_o

In [ ]:

# ----------------------------------------
# 5. Save outputs
# ----------------------------------------

# Save imputed data
imputed_data.to_csv("/content/drive/MyDrive/Train/data/vae_data_data.csv", index=False)


In [ ]:
# Combine with metadata and save df2
df2 = pd.concat([imputed_data, a[["TransactionID", "dataset_name"]]], axis=1)
df2.to_csv("/content/drive/MyDrive/Train/data/vae_index_index_data.csv", index=False)

print(" Imputation completed and saved.")


## Next


In [ ]:
# Impute missing values
missing_mask = data_normalized.isnull()
data_filled = data_normalized.fillna(0)
predicted_data = vae.predict(data_filled)
imputed_data = data_normalized.where(~missing_mask, predicted_data)
imputed_data = imputed_data * df_new.std() + df_new.mean()

# Round the imputed values to three decimal places
imputed_data = imputed_data.round(3)

# Display the imputed data
imputed_data

In [ ]:
df.shape

In [ ]:
imputed_data.shape

In [ ]:
aa=display(df)

counting list of Uniquic all featurs
isFraud  : 2 , [np.int64(0), np.int64(1)]
dataset_name  : 3 , ['Transaction_Fraud', 'Mobile_Transaction_Fraud']
transaction_type_EMI Payment  : 2 , [np.int64(0), np.int64(1)]
 Total ERC20 tnxs  : 300 , [np.int64(0), np.int64(265)]
oldbalanceOrg  : 54092 , [np.float64(0.0), np.float64(19391.0)]
transaction_type_Deposit  : 2 , [np.int64(0), np.int64(1)]
newbalanceOrig  : 42997 , [np.float64(0.0), np.float64(16374.82)]
 ERC20_most_rec_token_type_Others  : 2 , [np.int64(0), np.int64(1)]
merchant_category_Utilities  : 2 , [np.int64(0), np.int64(1)]
 ERC20 total ether sent  : 1415 , [np.float64(0.0), np.float64(35603169.52)]
 ERC20 most sent token type_Golem  : 2 , [np.int64(0), np.int64(1)]
device_used_Web  : 2 , [np.int64(1), np.int64(0)]
TransactionID  : 159841 , ['DigitalTrans_000001', 'DigitalTrans_000002']
transaction_type_Bill Payment  : 2 , [np.int64(1), np.int64(0)]
transaction_type_Loan Disbursement  : 2 , [np.int64(0), np.int64(1)]
merchant_cat

In [ ]:
bb=display(imputed_data)

counting list of Uniquic all featurs
isFraud  : 2 , [np.float64(0.0), np.float64(1.0)]
transaction_type_EMI Payment  : 2 , [np.float64(0.0), np.float64(1.0)]
 Total ERC20 tnxs  : 300 , [np.float64(0.0), np.float64(265.0)]
oldbalanceOrg  : 54092 , [np.float64(0.0), np.float64(19391.0)]
transaction_type_Deposit  : 2 , [np.float64(0.0), np.float64(1.0)]
newbalanceOrig  : 42997 , [np.float64(0.0), np.float64(16374.82)]
 ERC20_most_rec_token_type_Others  : 2 , [np.float64(0.0), np.float64(1.0)]
merchant_category_Utilities  : 2 , [np.float64(0.0), np.float64(1.0)]
 ERC20 total ether sent  : 1290 , [np.float64(0.0), np.float64(35603169.52)]
 ERC20 most sent token type_Golem  : 2 , [np.float64(0.0), np.float64(1.0)]
device_used_Web  : 2 , [np.float64(1.0), np.float64(-0.0)]
transaction_type_Bill Payment  : 2 , [np.float64(1.0), np.float64(0.0)]
transaction_type_Loan Disbursement  : 2 , [np.float64(0.0), np.float64(1.0)]
merchant_category_Groceries  : 2 , [np.float64(0.0), np.float64(1.0)]
 ERC

In [ ]:
print(a,b)

[2, 2, 300, 54092, 2, 42997, 2, 2, 1415, 2, 2, 2, 2, 2, 3460, 5854, 5717, 4, 2, 30267, 20, 2, 2, 2, 2, 2, 6728, 2, 2, 7810, 2, 2, 2, 2, 2, 258, 4719, 4, 2, 2, 727, 256, 107, 6223, 2, 2, 2647, 3, 2, 2, 6647, 4, 123, 641, 5013, 1309, 6302, 2, 6767, 1130, 70, 101058, 121, 2, 2, 459, 2, 2, 2, 2, 2, 2, 147, 2, 2, 5868, 4589, 4, 3380, 29, 61351, 57396, 476, 2, 1276, 2, 2] [2, 2, 300, 54092, 2, 42997, 2, 2, 1290, 2, 2, 2, 2, 2, 3190, 3903, 2942, 4, 2, 30267, 20, 2, 2, 2, 2, 2, 6194, 2, 2, 7810, 2, 2, 2, 2, 2, 258, 2148, 4, 2, 2, 727, 256, 107, 6223, 2, 2, 2424, 3, 2, 2, 5277, 4, 123, 641, 5013, 1296, 4989, 2, 4029, 1112, 70, 101054, 121, 2, 2, 459, 2, 2, 2, 2, 2, 2, 147, 2, 2, 5112, 2235, 4, 3113, 14, 61351, 57396, 444, 2, 900, 2, 2]


In [ ]:
print(b)

[2, 2, 300, 54092, 2, 42997, 2, 2, 1290, 2, 2, 2, 2, 2, 3190, 3903, 2942, 4, 2, 30267, 20, 2, 2, 2, 2, 2, 6194, 2, 2, 7810, 2, 2, 2, 2, 2, 258, 2148, 4, 2, 2, 727, 256, 107, 6223, 2, 2, 2424, 3, 2, 2, 5277, 4, 123, 641, 5013, 1296, 4989, 2, 4029, 1112, 70, 101054, 121, 2, 2, 459, 2, 2, 2, 2, 2, 2, 147, 2, 2, 5112, 2235, 4, 3113, 14, 61351, 57396, 444, 2, 900, 2, 2]


In [ ]:
print(imputed_data.isnull().sum())

isFraud                               0
transaction_type_EMI Payment          0
 Total ERC20 tnxs                     0
oldbalanceOrg                         0
transaction_type_Deposit              0
                                     ..
 ERC20 min val sent                   0
 ERC20 most sent token type_Others    0
 ERC20 min val rec                    0
type_CASH_OUT                         0
merchant_category_Entertainment       0
Length: 87, dtype: int64


In [ ]:
df2 = pd.concat([imputed_data, a[["TransactionID", "dataset_name"]]], axis=1)


In [ ]:
df2.to_csv("/content/drive/MyDrive/Assigments/vae_index_data.csv")

In [ ]:
imputed_data.to_csv("/content/drive/MyDrive/Assigments/vae_data_imputed_data.csv")

In [ ]:
from sklearn.model_selection import train_test_split

# Assuming 'imputed_data' is your full DataFrame
train_df, test_df = train_test_split(imputed_data, test_size=0.2, random_state=42)

# Optional: Check the size
print("Train shape:", train_df.shape)
print("Test shape:", test_df.shape)


Train shape: (127872, 87)
Test shape: (31969, 87)


In [ ]:
train_df.to_csv("/content/drive/MyDrive/Assigments/vae_train.csv")
test_df.to_csv("/content/drive/MyDrive/Assigments/vae_test.csv")

In [ ]:
# import pandas as pd

# # Ensure pandas doesn't truncate the column output
# pd.set_option('display.max_rows', None)

# # Then print the null counts
# print(df2.isnull().sum())


isFraud                                              0
transaction_type_EMI Payment                         0
 Total ERC20 tnxs                                    0
oldbalanceOrg                                        0
transaction_type_Deposit                             0
newbalanceOrig                                       0
 ERC20_most_rec_token_type_Others                    0
merchant_category_Utilities                          0
 ERC20 total ether sent                              0
 ERC20 most sent token type_Golem                    0
device_used_Web                                      0
transaction_type_Bill Payment                        0
transaction_type_Loan Disbursement                   0
merchant_category_Groceries                          0
 ERC20 total Ether received                          0
avg val sent                                         0
total ether balance                                  0
max val sent to contract                             0
transactio

In [ ]:
imputed_data['merchant_fraud_Kilback LLC'].value_counts()

,count
"WALLSMATERIAL_MODE_Stone, brick",
0.211,50000
0.000,39446
1.000,10554


In [ ]:
imputed_data1=imputed_data.iloc[:,:30]

In [ ]:
imputed_data

In [ ]:
imputed_data1.to_csv('test.csv')

In [ ]:
imputed_data_round = imputed_data.round(0)
imputed_data_round

,amt,lat,long,city_pop,merch_lat,merch_long,merchant_fraud_Cormier LLC,merchant_fraud_Kilback LLC,merchant_fraud_Kuhn LLC,merchant_fraud_Schumm PLC,...,HOUSETYPE_MODE_block of flats,HOUSETYPE_MODE_specific housing,HOUSETYPE_MODE_terraced house,WALLSMATERIAL_MODE_Mixed,WALLSMATERIAL_MODE_Panel,"WALLSMATERIAL_MODE_Stone, brick",WALLSMATERIAL_MODE_Wooden,EMERGENCYSTATE_MODE_Others,EMERGENCYSTATE_MODE_Yes,is_fraud
0,70.0,39.0,-90.0,88416.0,39.0,-90.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,70.0,39.0,-90.0,88416.0,39.0,-90.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,70.0,39.0,-90.0,88416.0,39.0,-90.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,70.0,39.0,-90.0,88416.0,39.0,-90.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,70.0,39.0,-90.0,88416.0,39.0,-90.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,3.0,37.0,-91.0,241.0,36.0,-91.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
99996,63.0,41.0,-74.0,5950.0,41.0,-74.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
99997,52.0,27.0,-82.0,3776.0,26.0,-82.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
99998,3.0,35.0,-99.0,540.0,35.0,-98.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
imputed_data_round['merchant_fraud_Kilback LLC'].value_counts()

,count
merchant_fraud_Kilback LLC,
0.0,99829
1.0,171


In [ ]:
imputed_data_round.to_csv('cred+simucred_vae.csv')

In [ ]:
dff=pd.read_csv("C:\Users\adars\final_data_vae.csv")

In [ ]:
for i in imputed_data_round.columns:
    print(i)

In [ ]:
import numpy as np
import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd

# Load your dataset
# df = pd.read_csv('your_dataset.csv')  # Load your DataFrame here if not already loaded

# Convert DataFrame to a numpy array
data = df_new.to_numpy()

# Convert to PyTorch tensors
data_tensor = torch.tensor(data, dtype=torch.float32)

# Create DataLoader
batch_size = 32
dataset = TensorDataset(data_tensor, data_tensor)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# VAE Model
class Encoder(nn.Module):
    def __init__(self, input_dim, hidden_dim, latent_dim):
        super(Encoder, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.fc_mu = nn.Linear(hidden_dim, latent_dim)
        self.fc_logvar = nn.Linear(hidden_dim, latent_dim)

    def forward(self, x):
        h = F.relu(self.fc1(x))
        mu = self.fc_mu(h)
        logvar = self.fc_logvar(h)
        return mu, logvar

class Decoder(nn.Module):
    def __init__(self, latent_dim, hidden_dim, output_dim):
        super(Decoder, self).__init__()
        self.fc1 = nn.Linear(latent_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, z):
        h = F.relu(self.fc1(z))
        x_reconstructed = torch.sigmoid(self.fc2(h))
        return x_reconstructed

class VAE(nn.Module):
    def __init__(self, input_dim, hidden_dim, latent_dim):
        super(VAE, self).__init__()
        self.encoder = Encoder(input_dim, hidden_dim, latent_dim)
        self.decoder = Decoder(latent_dim, hidden_dim, input_dim)

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std

    def forward(self, x):
        mu, logvar = self.encoder(x)
        z = self.reparameterize(mu, logvar)
        x_reconstructed = self.decoder(z)
        return x_reconstructed, mu, logvar

def vae_loss(reconstructed_x, x, mu, logvar):
    reconstruction_loss = F.mse_loss(reconstructed_x, x, reduction='sum')
    kl_divergence = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return reconstruction_loss + kl_divergence

# Training
input_dim = data.shape[1]
hidden_dim = 64
latent_dim = 2
vae = VAE(input_dim, hidden_dim, latent_dim)
optimizer = torch.optim.Adam(vae.parameters(), lr=1e-3)

num_epochs = 100
vae.train()
for epoch in range(num_epochs):
    for batch in dataloader:
        x, _ = batch
        optimizer.zero_grad()
        reconstructed_x, mu, logvar = vae(x)
        loss = vae_loss(reconstructed_x, x, mu, logvar)
        loss.backward()
        optimizer.step()

    print(f'Epoch {epoch+1}, Loss: {loss.item()}')

# Generating Synthetic Data
vae.eval()
with torch.no_grad():
    z = torch.randn(100, latent_dim)
    generated_samples = vae.decoder(z)
    generated_samples = generated_samples.numpy()
    print(generated_samples)


In [ ]:
data=df_new

In [ ]:
# import numpy as np
# import pandas as pd
# import tensorflow as tf
# from tensorflow.keras.layers import Input, Dense, Lambda
# from tensorflow.keras.models import Model
# from tensorflow.keras import backend as K
# from sklearn.preprocessing import MinMaxScaler

# # Example DataFrame
# # Replace this with your own dataset loading and preprocessing steps

# # Scale the data
# scaler = MinMaxScaler()
# data_scaled = scaler.fit_transform(df_new)

# # Define the VAE architecture
# original_dim = data.shape[1]
# intermediate_dim = 10
# latent_dim = 2
# batch_size = 10
# epochs = 50

# # Encoder
# inputs = Input(shape=(original_dim,))
# h = Dense(intermediate_dim, activation='relu')(inputs)
# z_mean = Dense(latent_dim)(h)
# z_log_var = Dense(latent_dim)(h)

# # Sampling function
# def sampling(args):
#     z_mean, z_log_var = args
#     epsilon = K.random_normal(shape=(K.shape(z_mean)[0], latent_dim))
#     return z_mean + K.exp(0.5 * z_log_var) * epsilon

# # Reparameterization trick
# z = Lambda(sampling, output_shape=(latent_dim,))([z_mean, z_log_var])

# # Decoder
# decoder_h = Dense(intermediate_dim, activation='relu')
# decoder_mean = Dense(original_dim, activation='sigmoid')
# h_decoded = decoder_h(z)
# x_decoded_mean = decoder_mean(h_decoded)

# # VAE model
# vae = Model(inputs, x_decoded_mean)

# # Loss function
# xent_loss = original_dim * tf.keras.losses.binary_crossentropy(inputs, x_decoded_mean)
# kl_loss = -0.5 * tf.reduce_sum(1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var), axis=-1)
# vae_loss = tf.reduce_mean(xent_loss + kl_loss)

# vae.add_loss(vae_loss)
# vae.compile(optimizer='adam')

# # Training
# vae.fit(data_scaled, epochs=epochs, batch_size=batch_size, verbose=0)

# # Impute missing values using the trained VAE
# data_imputed_scaled = vae.predict(data_scaled)

# # Inverse transform scaling
# data_imputed = scaler.inverse_transform(data_imputed_scaled)

# # Replace the missing values in original data with imputed values
# data_filled = data.copy()
# data_filled.values[missing_mask] = data_imputed[missing_mask]

# # Now data_filled contains your DataFrame with missing values imputed using VAE

# # Example of how to use the imputed data
# print("Original Data:")
# print(data)
# print("\nData with Missing Values:")
# print(data_missing)
# print("\nImputed Data:")
# print(data_filled)
